In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import random
import pandas as pd


In [ ]:
LR = 0.001

In [ ]:
data_path = "data.csv"

In [ ]:
data = np.loadtxt(data_path, delimiter=',')

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.labels = self.data.iloc[:, -1].values.astype(int)
        self.features = self.data.iloc[:, :-1].values.astype(float).reshape(-1, 30, 30)
        
    def __len__(self):
        return len(self.labels)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # Convert to PyTorch tensors
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return feature, label

In [ ]:
class SpaceRatNetwork(nn.Module):
    def __init__(self):
        super(SpaceRatNetwork, self).__init__()
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(30*30, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x 
        

In [ ]:
dataset = CustomDataset(data_path)
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
# for inputs, labels in dataloader:
#     print(f"Inputs: {inputs.shape}")  # Should be [batch_size, 30, 30]
#     print(f"Labels: {labels.shape}")  # Should be [batch_size]
#     break

# Train-Test Split (80% train, 20% test)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)





In [ ]:
model = SpaceRatNetwork()
loss_funtion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
model.train()


num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute the loss
        loss = criterion(outputs.squeeze(), targets)
        train_loss += loss.item()
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}")

# Testing Loop
model.eval()
test_loss = 0.0
with torch.no_grad():  # No need to compute gradients during testing
    for inputs, targets in test_loader:
        # Forward pass
        outputs = model(inputs)
        
        # Compute the loss
        loss = loss_funtion(outputs.squeeze(), targets)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
print(f"Test Loss: {avg_test_loss:.4f}")